In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
#import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math
import dill as pickle
from copy import deepcopy
from collections import Counter

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from mcmodels.core.utils import get_ccf_data

from mcmodels.core import VoxelModelCache
from mcmodels.connectivity.creation import get_connectivity_matrices3
from mcmodels.core.connectivity_data import get_connectivity_data
#from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures, get_leaves_ontologicalorder, get_indices_2ormore, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss, Crossval
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists,CrossvalNW

/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#read data
TOP_DIR = workingdirectory
INPUT_JSON = workingdirectory + '/data/data_final/input_011520.json'
EXPERIMENTS_EXCLUDE_JSON = workingdirectory + '/data/data_final/experiments_exclude.json'
input_data = ju.read(INPUT_JSON)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
cache = VoxelModelCache(manifest_file=manifest_file)
st = cache.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}
major_structures = np.load(workingdirectory + '/analyses/paper_final/metadata/major_structures.npy')
major_structure_ids = np.load(workingdirectory + '/analyses/paper_final/metadata/major_structure_ids.npy')
data_info = pd.read_excel(workingdirectory + '/data/data_final/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
ontological_order_leaves = np.load(workingdirectory + '/analyses/paper_final/metadata/ontological_order_leaves_v3.npy')
COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]

annotation/ccf_2017


In [3]:
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False, structure_set_id=167587189)
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()


512
703
1089
1097
315
313
354
698
771
803
477
549


In [4]:
ontological_order = get_ordered_summary_structures(cache,167587189)
#due to redundancy in ccfv3, remove 'MDRNv', 'MDRNd' from summary structures (include as leafs of MDRN)
ontological_order[277]
ontological_order = np.setdiff1d(ontological_order, [1098, 1107])


In [5]:
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)
connectivity_data.leafs = leafs
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)
sid0 = list(connectivity_data.structure_datas.keys())[0]
eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
# contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
# ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)
# ipsi_target_regions, ipsi_target_counts = nonzero_unique(ipsi_targetkey, return_counts=True)
# contra_target_regions, contra_target_counts = nonzero_unique(contra_targetkey, return_counts=True)
# target_order = lambda x: np.array(ontological_order)[np.isin(ontological_order, x)]
# permutation = lambda x: np.argsort(np.argsort(target_order(x)))
# targ_ids = np.concatenate([ipsi_target_regions[permutation(ipsi_target_regions)],
#                            contra_target_regions[permutation(contra_target_regions)]])
targ_ord = ontological_order_leaves
source_ord = ontological_order_leaves
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=targ_ord, hemisphere_id=1)
#ontological_order = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_v3.npy')
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=targ_ord, hemisphere_id=2)


connectivity_data.get_regionalized_normalized_data(ontological_order, ipsi_targetkey, contra_targetkey)

above_thresh = {}
thresh = 1
for m in range(12):
    sid = major_structure_ids[m]
    above_thresh[sid] = np.where(connectivity_data.structure_datas[sid].reg_inj.sum(axis = 1) > thresh)[0]

In [6]:
for sid in major_structure_ids:
    connectivity_data.structure_datas[sid].reg_proj_injnorm = connectivity_data.structure_datas[sid].reg_proj_injnorm[above_thresh[sid]]
    connectivity_data.structure_datas[sid].centroids = connectivity_data.structure_datas[sid].centroids[above_thresh[sid]]
    connectivity_data.creline[sid] = connectivity_data.creline[sid][above_thresh[sid]]
    connectivity_data.leafs[sid] = connectivity_data.leafs[sid][above_thresh[sid]]

gammas = np.asarray([0.0001,0.001,0.01,0.1,1,10,100,1000,10000])
reg_proj_injnorm = {sid: connectivity_data.structure_datas[sid].reg_proj_injnorm for sid in major_structure_ids}
crelines = {sid: connectivity_data.creline[sid] for sid in major_structure_ids}
connectivity_data.leaf2_index_matrices = get_indices_2ormore(connectivity_data.leafs)
connectivity_data.get_creleaf_combos()
connectivity_data.creleaf2_index_matrices = get_indices_2ormore(connectivity_data.creleaf_combos)
connectivity_data.get_creleaf_combos()
connectivity_data.cre2_index_matrices = get_indices_2ormore(crelines)
connectivity_data.creleaf2_evalindices = get_eval_indices(connectivity_data.creleaf2_index_matrices)
creleaf2_evalindices = {sid: connectivity_data.creleaf2_evalindices[sid] for sid in major_structure_ids} 
cre2_index_matrices = {sid: connectivity_data.cre2_index_matrices[sid] for sid in major_structure_ids} 
connectivity_data.major2_index_matrices = {}
for sid in major_structure_ids:
     connectivity_data.major2_index_matrices[sid] =  np.expand_dims(np.ones(crelines[sid].shape),0)
major2_index_matrices = {sid:connectivity_data.major2_index_matrices[sid] for sid in major_structure_ids}

wtmajor2_index_matrices = {}
for sid in major_structure_ids:
    wtm = np.zeros(major2_index_matrices[sid].shape) 
    #print(wtm.shape)
    wtmajor2_index_matrices[sid] = wtm[:,np.where(crelines[sid] == 'C57BL/6J')[0]] = 1
#get_indices2(connectivity_data.creline[sid], np.asarray(['C57BL/6J']),leafs[sid])

#ncresum2_leaf2 = np.zeros(12)
#ncresum2 = np.zeros(12)
ncreleaf2 = np.zeros(12)
ntotal = np.zeros(12)
for i in range(12):
    sid = major_structure_ids[i]
    ntotal[i] = connectivity_data.structure_datas[sid].reg_proj.shape[0]
    #ncresum2_leaf2[i] =len(cresum2_leaf2_evalindices[sid])
    #ncresum2[i] = len(cresum2_evalindices[sid])
    ncreleaf2[i] = len(creleaf2_evalindices[sid])

major_structures = np.asarray(major_structures)
reo = np.asarray([4,7,2,1,10,9,11,3,5,8,6,0], dtype = int)
df = pd.DataFrame([ntotal,  ncreleaf2], dtype = int, index=['Total','Cre-Leaf'], columns = major_structures[reo]).transpose()
#df = pd.DataFrame([ntotal, ncresum2, ncresum2_leaf2,  ncreleaf2], dtype = int, index=['Total','Cre-Summary','Cre-Summary, Leaf','Cre-Leaf']).transpose()
print(df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Total &  Cre-Leaf \\
\midrule
Isocortex &     36 &         4 \\
OLF       &      7 &         2 \\
HPF       &    122 &        59 \\
CTXsp     &     85 &        38 \\
STR       &   1128 &       709 \\
PAL       &     68 &        18 \\
TH        &     46 &         7 \\
HY        &     35 &        17 \\
MB        &     33 &         8 \\
P         &     30 &        11 \\
MY        &     78 &        44 \\
CB        &     83 &        29 \\
\bottomrule
\end{tabular}



In [7]:

major2_index_matrices = {sid:connectivity_data.major2_index_matrices[sid] for sid in major_structure_ids}

In [8]:
major2_index_matrices[315].shape

(1, 1102)

In [11]:
connectivity_data.cre2_index_matrices[315].shape

(51, 1102)

In [12]:
#get surfaces leaf
frac_learn = np.ones(12)
for m in range(12):
    sid = major_structure_ids[m]
    connectivity_data.structure_datas[sid].loss_surface_cv_leaf = get_loss_surface_cv_spline(connectivity_data.structure_datas[sid].reg_proj_norm,
                                                                                                 connectivity_data.structure_datas[sid].centroids,
                                                                                                 connectivity_data.creline[sid],
                                                                                                 connectivity_data.leafs[sid],
                                                                                                 frac_learn[m])
    connectivity_data.structure_datas[sid].smoothed_losses_leaf = get_embedding_cv(surface = connectivity_data.structure_datas[sid].loss_surface_cv_leaf,
                                                                              dists = pairwise_distances(connectivity_data.structure_datas[sid].centroids)**2,
                                                                              cre_distances_cv = connectivity_data.structure_datas[sid].loss_surface_cv_leaf.cre_distances_cv)
    

/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/samsonkoelle/mouse_connectivity_models/mcmodels/models/expectedloss/shapeconstrained.py:47: RuntimeWarning: invalid value encountered in true_divide
  coordinates_normed = coordinates / np.linalg.norm(coordinates, axis=0)  # **2
/Users/s

In [13]:
reg_proj_norm = {sid: connectivity_data.structure_datas[sid].reg_proj_norm[above_thresh[sid]] for sid in major_structure_ids}
creleaf2_index_matrices = {sid: connectivity_data.creleaf2_index_matrices[sid] for sid in major_structure_ids} 
leaf2_index_matrices = {sid: connectivity_data.leaf2_index_matrices[sid] for sid in major_structure_ids} 
pds = {}
for sid in major_structure_ids:
    pds[sid] = pairwise_distances(connectivity_data.structure_datas[sid].centroids)**2
distances = pds

In [ ]:
##{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_major_creleaf2 = CrossvalNW(reg_proj_norm, distances,major2_index_matrices, creleaf2_evalindices,gammas)
nw_major_creleaf2.gammas = gammas
nw_major_creleaf2.predictions = nw_major_creleaf2.get_predictions()
nw_major_creleaf2.get_results_loocv()
nw_major_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [28]:
np.where(model_index_val == eval_index_val[i])[0]

array([1101])

In [30]:
print(reg_proj_norm[315].shape)
print(distances[315].shape)
print(major2_index_matrices[315].shape)

(1102, 1123)
(1102, 1102)
(1, 1102)


In [33]:
model_indices.shape

(1102,)

In [34]:
eval_index_val = np.where(eval_indices == 1)[0]
model_index_val = np.where(model_indices == 1)[0]


In [35]:
model_index_val

array([   0,    1,    2, ..., 1099, 1100, 1101])

In [36]:
print(model_indices.shape)

(1102,)


In [26]:
projections = reg_proj_norm[315]
dists = distances[315]
gamma = .5
model_indices = major2_index_matrices[315][0]
eval_indices = major2_index_matrices[315][0]

eval_index_val = np.where(eval_indices == 1)[0]
model_index_val = np.where(model_indices == 1)[0]
#print('e', eval_index_val, 'm', model_index_val)
projections = np.asarray(projections, dtype=np.float32)
nmod_ind = len(model_index_val)
neval = len(eval_index_val)
# nexp = centroids.shape[0]
predictions = np.empty(projections.shape)
predictions[:] = np.nan
if len(model_index_val) > 0 and len(eval_index_val) > 0:
    # weights = np.exp(-dists[model_index_val][:, eval_index_val] / gamma)#np.exp(-dists[model_index_val] / gamma) #get_weights(centroids, gamma)
    for i in range(neval):
        matchindex = np.where(model_index_val == eval_index_val[i])[0]
        otherindices = np.setdiff1d(np.asarray(list(range(nmod_ind))), matchindex)
        # this order of operations is the fastest I found
        dists_i = dists[model_index_val][:, eval_index_val[i]] - np.min(
            dists[model_index_val[otherindices]][:, eval_index_val[i]])
        weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])
        # print(np.nansum(weights[:,i][otherindices]))
        weights_i[matchindex] = 0
        weights_i = np.asarray(weights_i, dtype=np.float32)
        weights_i = weights_i / np.sum(weights_i)
        # weights_i[np.isnan(weights_i)] = 0.
        pred = np.dot(weights_i, projections[model_index_val])
        #pred = pred / np.linalg.norm(pred)
        predictions[eval_index_val[i]] = pred


In [ ]:
def get_nwloocv_predictions_singlemodel_dists(projections, dists, gamma, model_indices, eval_indices):

    eval_index_val = np.where(eval_indices == 1)[0]
    model_index_val = np.where(model_indices == 1)[0]
    #print('e', eval_index_val, 'm', model_index_val)
    projections = np.asarray(projections, dtype=np.float32)
    nmod_ind = len(model_index_val)
    neval = len(eval_index_val)
    # nexp = centroids.shape[0]
    predictions = np.empty(projections.shape)
    predictions[:] = np.nan
    if len(model_index_val) > 0 and len(eval_index_val) > 0:
        # weights = np.exp(-dists[model_index_val][:, eval_index_val] / gamma)#np.exp(-dists[model_index_val] / gamma) #get_weights(centroids, gamma)
        for i in range(neval):
            matchindex = np.where(model_index_val == eval_index_val[i])[0]
            otherindices = np.setdiff1d(np.asarray(list(range(nmod_ind))), matchindex)
            # this order of operations is the fastest I found
            dists_i = dists[model_index_val][:, eval_index_val[i]] - np.min(
                dists[model_index_val[otherindices]][:, eval_index_val[i]])
            weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])
            # print(np.nansum(weights[:,i][otherindices]))
            weights_i[matchindex] = 0
            weights_i = np.asarray(weights_i, dtype=np.float32)
            weights_i = weights_i / np.sum(weights_i)
            # weights_i[np.isnan(weights_i)] = 0.
            pred = np.dot(weights_i, projections[model_index_val])
            #pred = pred / np.linalg.norm(pred)
            predictions[eval_index_val[i]] = pred

    return (predictions)

In [22]:
#distances = pds#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_creleaf_creleaf2 = CrossvalNW(reg_proj_norm, distances,creleaf2_index_matrices, creleaf2_evalindices,gammas)
nw_creleaf_creleaf2.gammas = gammas# ds
nw_creleaf_creleaf2.predictions = nw_creleaf_creleaf2.get_predictions()
nw_creleaf_creleaf2.get_results_loocv()
nw_creleaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


/Users/samsonkoelle/mouse_connectivity_models/mcmodels/models/voxel/crossvalidation.py:43: RuntimeWarning: overflow encountered in exp
  weights_i = np.exp(-dists_i * gamma)  # weights[i,:] / np.nansum(weights[i,:][otherindices])


In [23]:
nw_creleaf_creleaf2.meanloss

array([0.06219172, 0.62132425, 0.14592686, 0.34548913, 0.18368028,
       0.14053769, 0.25252631, 0.16467688, 0.29210916, 0.20105935,
       0.0965846 , 0.61511732])

In [24]:

#distances = pds#{sid: connectivity_data.structure_datas[sid].smoothed_losses_str for sid in major_structure_ids}
nw_cremajor_creleaf2 = CrossvalNW(reg_proj_norm, distances,cre2_index_matrices, creleaf2_evalindices,gammas)
nw_cremajor_creleaf2.gammas = gammas# ds
nw_cremajor_creleaf2.predictions = nw_cremajor_creleaf2.get_predictions()
nw_cremajor_creleaf2.get_results_loocv()
nw_cremajor_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)

mean_gammas = np.ones(12)*0.000001
mean_creleaf_creleaf2 = CrossvalNW(reg_proj_norm, distances, creleaf2_index_matrices, creleaf2_evalindices,mean_gammas)
mean_creleaf_creleaf2.gammas = mean_gammas# ds
mean_creleaf_creleaf2.predictions = mean_creleaf_creleaf2.get_predictions()
mean_creleaf_creleaf2.get_results_loocv()
mean_creleaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


In [25]:
nw_creleaf_creleaf2.meanloss_weighted

array([0.06219172, 0.62132425, 0.1728102 , 0.35197502, 0.22421699,
       0.16053243, 0.26838125, 0.19121029, 0.29210916, 0.20387396,
       0.1249916 , 0.67317245])

In [26]:

nw_majorwt_creleaf2 = CrossvalNW(reg_proj_norm, distances,wtmajor2_index_matrices, creleaf2_evalindices,gammas)
nw_majorwt_creleaf2.gammas = gammas
nw_majorwt_creleaf2.predictions = nw_major_creleaf2.get_predictions()
nw_majorwt_creleaf2.get_results_loocv()
nw_majorwt_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)


# nw_sum_cresum2leaf2 = CrossvalNW(reg_proj_norm, distances,sum2_index_matrices, cresum2_leaf2_evalindices,gammas)
# nw_sum_cresum2leaf2.gammas = gammas
# nw_sum_cresum2leaf2.predictions = nw_sum_cresum2leaf2.get_predictions()
# nw_sum_cresum2leaf2.get_results_loocv()
# nw_sum_cresum2leaf2.get_results_weightedloocv(connectivity_data.summary_structures,crelines,ia_map)
#nw_sum_creleaf2.meanloss

nw_leaf_creleaf2 = CrossvalNW(reg_proj_norm, distances,leaf2_index_matrices, creleaf2_evalindices,gammas)
nw_leaf_creleaf2.gammas = gammas# ds
nw_leaf_creleaf2.predictions = nw_leaf_creleaf2.get_predictions()
nw_leaf_creleaf2.get_results_loocv()
nw_leaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)



In [27]:
leaf2_index_matrices = {sid: connectivity_data.leaf2_index_matrices[sid] for sid in major_structure_ids} 
distances = {sid: connectivity_data.structure_datas[sid].smoothed_losses_leaf for sid in major_structure_ids}
twostage_leaf_creleaf2 = CrossvalNW(reg_proj_norm, distances, leaf2_index_matrices, creleaf2_evalindices,gammas)
twostage_leaf_creleaf2.gammas = gammas #ds
twostage_leaf_creleaf2.predictions = twostage_leaf_creleaf2.get_predictions()
twostage_leaf_creleaf2.get_results_loocv()
twostage_leaf_creleaf2.get_results_weightedloocv(connectivity_data.leafs,crelines,ia_map)

In [28]:
#el_labels = np.zeros(150, dtype = str)
models = np.asarray(['Mean','NW','NW','NW','NW','NW','EL'])
datas = np.asarray([r"$I_c \cap I_L$",
                   r"$I_c \cap I_M$",
                   r"$I_c \cap I_L$",
                    r"$I_L$",
                  r"$I_{wt} \cap I_M$",
                  r"$I_M$"  ,
                  r"$I_L$"])
multi_ind = np.vstack([models,datas])
multi_ind = np.asarray(multi_ind, dtype = str)
multi_ind = list(zip(*multi_ind))
multi_ind = pd.MultiIndex.from_tuples(multi_ind, names=[r"$\widehat f$",r"$\mathcal D$"])

reo = np.asarray([4,7,2,1,10,9,11,3,5,8,6,0])
results = pd.DataFrame([mean_creleaf_creleaf2.meanloss_weighted,
nw_cremajor_creleaf2.meanloss_weighted,
nw_creleaf_creleaf2.meanloss_weighted,
nw_leaf_creleaf2.meanloss_weighted,
nw_majorwt_creleaf2.meanloss_weighted,
nw_major_creleaf2.meanloss_weighted,
twostage_leaf_creleaf2.meanloss_weighted], index = multi_ind, columns = major_structures).transpose()#, cre2_index_matrices
trunc = lambda x: math.trunc(1000 * x) / 1000;

results = results.applymap(trunc)

In [29]:
print(results.iloc[reo].to_latex(index=True,escape=False))

\begin{tabular}{lrrrrrrr}
\toprule
$\widehat f$ &           Mean & \multicolumn{5}{l}{NW} &     EL \\
$\mathcal D$ & $I_c \cap I_L$ & $I_c \cap I_M$ & $I_c \cap I_L$ &  $I_L$ & $I_{wt} \cap I_M$ &  $I_M$ &  $I_L$ \\
\midrule
Isocortex &          0.229 &          0.248 &          0.224 &  0.274 &             0.269 &  0.269 &  0.217 \\
OLF       &          0.193 &          0.233 &          0.191 &  0.135 &             0.179 &  0.179 &  0.138 \\
HPF       &          0.178 &          0.342 &          0.172 &  0.212 &             0.235 &  0.235 &  0.172 \\
CTXsp     &          0.621 &          0.621 &          0.621 &  0.621 &             0.621 &  0.621 &  0.621 \\
STR       &          0.128 &          0.117 &          0.124 &  0.171 &             0.234 &  0.234 &  0.125 \\
PAL       &          0.203 &          0.205 &          0.203 &  0.295 &             0.291 &  0.291 &  0.188 \\
TH        &          0.673 &          0.664 &          0.673 &  0.358 &             0.379 &  0.379 &  0.417 \